Using a suitable program demonstrate the working of RNN in a classification task for temporal sequences.

Example Used: Sentiment Analysis

DataSet

In [0]:
train_data = {
  'good': True,
  'bad': False,
  'happy': True,
  'sad': False,
  'not good': False,
  'not bad': True,
  'not happy': False,
  'not sad': True,
  'very good': True,
  'very bad': False,
  'very happy': True,
  'very sad': False,
  'i am happy': True,
  'this is good': True,
  'i am bad': False,
  'this is bad': False,
  'i am sad': False,
  'this is sad': False,
  'i am not happy': False,
  'this is not good': False,
  'i am not bad': True,
  'this is not sad': True,
  'i am very happy': True,
  'this is very good': True,
  'i am very bad': False,
  'this is very sad': False,
  'this is very happy': True,
  'i am good not bad': True,
  'this is good not bad': True,
  'i am bad not good': False,
  'i am good and happy': True,
  'this is not good and not happy': False,
  'i am not at all good': False,
  'i am not at all bad': True,
  'i am not at all happy': False,
  'this is not at all sad': True,
  'this is not at all happy': False,
  'i am good right now': True,
  'i am bad right now': False,
  'this is bad right now': False,
  'i am sad right now': False,
  'i was good earlier': True,
  'i was happy earlier': True,
  'i was bad earlier': False,
  'i was sad earlier': False,
  'i am very bad right now': False,
  'this is very good right now': True,
  'this is very sad right now': False,
  'this was bad earlier': False,
  'this was very good earlier': True,
  'this was very bad earlier': False,
  'this was very happy earlier': True,
  'this was very sad earlier': False,
  'i was good and not bad earlier': True,
  'i was not good and not happy earlier': False,
  'i am not at all bad or sad right now': True,
  'i am not at all good or happy right now': False,
  'this was not happy and not good earlier': False,
}

test_data = {
  'this is happy': True,
  'i am good': True,
  'this is not happy': False,
  'i am not good': False,
  'this is not bad': True,
  'i am not sad': True,
  'i am very good': True,
  'this is very bad': False,
  'i am very sad': False,
  'this is bad not good': False,
  'this is good and happy': True,
  'i am not good and not happy': False,
  'i am not at all sad': True,
  'this is not at all good': False,
  'this is not at all bad': True,
  'this is good right now': True,
  'this is sad right now': False,
  'this is very bad right now': False,
  'this was good earlier': True,
  'i was not happy and not good earlier': False,
}

Pre-processing to construct a vocabulary of all words that exist in our data:

In [0]:
vocab = list(set([w for text in train_data.keys() for w in text.split(' ')]))
vocab_size = len(vocab)
print('%d unique words found' % vocab_size) # 18 unique words found

18 unique words found


In [0]:
# Assign indices to each word.
# RNNs can’t understand words - we have to give them numbers.
word_to_idx = { w: i for i, w in enumerate(vocab) }
idx_to_word = { i: w for i, w in enumerate(vocab) }
print(word_to_idx['good']) # 0 (this may change)
print(idx_to_word[0]) # good (this may change)

0
good


Each input $x_{i}$ to our RNN is a vector. We’ll use **one-hot vectors**, which contain all zeros except for a single one.

Since we have 18 unique words in our vocabulary, each $x_{i}$ will be a 18-dimensional one-hot vector

In [0]:
import numpy as np

def createInputs(text):
  '''
  Returns an array of one-hot vectors representing the words
  in the input text string.
  - text is a string
  - Each one-hot vector has shape (vocab_size, 1)
  '''
  inputs = []
  for w in text.split(' '):
    v = np.zeros((vocab_size, 1))
    v[word_to_idx[w]] = 1
    inputs.append(v)
  return inputs

**Forward Propagation**


$h^{\langle t \rangle} = \tanh(W_{hh} h^{\langle t-1 \rangle} + W_{xh} x^{\langle t \rangle} + b_h)$.

$\hat{y}^{\langle t \rangle} = softmax(W_{hy} h^{\langle t \rangle} + b_y)$.


In [0]:
import numpy as np
from numpy.random import randn

np.random.seed(1)
##
class RNN:
  # A Vanilla Recurrent Neural Network.

  def __init__(self, input_size, output_size, hidden_size=64):
    # Weights
    self.Whh = randn(hidden_size, hidden_size) / 1000
    self.Wxh = randn(hidden_size, input_size) / 1000
    self.Why = randn(output_size, hidden_size) / 1000

    # Note: We're dividing by 1000 to reduce the initial variance of our weights.
    # This is not the best way to initialize weights, but it's simple and works for this example.

    # Biases
    self.bh = np.zeros((hidden_size, 1))
    self.by = np.zeros((output_size, 1))

  def forward(self, inputs):
    '''
    Perform a forward pass of the RNN using the given inputs.
    Returns the final output and hidden state.
    - inputs is an array of one hot vectors with shape (input_size, 1).
    '''
    h = np.zeros((self.Whh.shape[0], 1))

    self.last_inputs = inputs
    self.last_hs = { 0: h }

    # Perform each step of the RNN
    for i, x in enumerate(inputs):
      h = np.tanh(self.Wxh @ x + self.Whh @ h + self.bh)
      self.last_hs[i + 1] = h

    # Compute the output
    y = self.Why @ h + self.by

    return y, h 
  ##
    

In [0]:
def softmax(xs):
  # Applies the Softmax Function to the input array.
  return np.exp(xs) / sum(np.exp(xs))

# Initialize our RNN!
rnn = RNN(vocab_size, 2)


inputs = createInputs('i am very good')
out, h = rnn.forward(inputs)
probs = softmax(out)
print(probs) # [[0.50000095], [0.49999905]]

[[0.49999766]
 [0.50000234]]


**Back Propagation**

We’ll use cross-entropy loss, which is often paired with Softmax. Here’s how we calculate it:

L = $-ln ({p_c})$  

where ${p_c}$ is our RNN’s predicted probability for the correct class (positive or negative).

**Gradients**

We’ll start by calculating $\frac{\partial L}{\partial y}$. We know:

$L = -\ln(p_c) = -\ln(\text{softmax}(y_c))$

Using Chain Rule:

$\frac{\partial L}{\partial y}=\{ {p_i \space\space\space\space if \space\space i \space != \space c \space\space\space\space}, { \space\space\space\space p_i -1 \space\space\space\space if \space\space i \space = \space c}\}$

For example, if we have p = [0.2, 0.2, 0.6] and the correct class is c = 0, then we’d get $\frac{\partial L}{\partial y}$ = [-0.8, 0.2, 0.6] 

$\frac{\partial L}{\partial W_{hy}} = \frac{\partial L}{\partial y} *  \frac{\partial y}{\partial W_{hy}}$
      
${y = W_{hy} h_n + b_y}$

​where $h_n$ is the final hidden state. Thus,

$\frac{\partial y}{\partial W_{hy}} = {h_n} $
​	
 $\frac{\partial L}{\partial W_{hy}} = \boxed{\frac{\partial L}{\partial y} h_n}$
 
Similarly,

$\frac{\partial y}{\partial b_y} = 1$

$\frac{\partial L}{\partial b_y} = \boxed{\frac{\partial L}{\partial y}}$

Finally, we need the gradients for $W_{hh}, W_{xh},$ and $b_h$, which are used every step during the RNN. 

We have:

$\frac{\partial L}{\partial W_{xh}} = \frac{\partial L}{\partial y} \sum_t \frac{\partial y}{\partial h_t} * \frac{\partial h_t}{\partial W_{xh}}$

​because changing $W_{xh}$ affects every $h_t$, which all affect y and ultimately L. In order to fully calculate the gradient of $W_{xh}$ , we’ll need to backpropagate through all timesteps, which is known as **Backpropagation Through Time (BPTT)**

$W_{xh}$ is used for all $x_t$ → $h_t$ forward links, so we have to backpropagate back to each of those links.

Once we arrive at a given step t, we need to calculate $\frac{\partial h_t}{\partial W_{xh}}$ :

$h_t = \tanh (W_{xh} x_t + W_{hh} h_{t-1} + b_h)$

The derivative of $\tanh$ is well-known:

$\frac{d \tanh(x)}{dx} = 1 - \tanh^2(x)$

We use Chain Rule like usual:

$\frac{\partial h_t}{\partial W_{xh}} = \boxed{(1 - h_t^2) x_t}$
 
Similarly, 

$\frac{\partial h_t}{\partial W_{hh}} = \boxed{(1 - h_t^2) h_{t-1}}$

$\frac{\partial h_t}{\partial b_h} = \boxed{(1 - h_t^2)}$

 
The last thing we need is $\frac{\partial y}{\partial h_t} $. We can calculate this recursively:

$\begin{aligned} \frac{\partial y}{\partial h_t} &= \frac{\partial y}{\partial h_{t+1}} * \frac{\partial h_{t+1}}{\partial h_t} \\ &= \frac{\partial y}{\partial h_{t+1}} (1 - h_t^2) W_{hh} \\ \end{aligned}$

We’ll implement BPTT starting from the last hidden state and working backwards, so we’ll already have  $\frac{\partial y}{\partial h_{t+1}}$ by the time we want to calculate $\frac{\partial y}{\partial h_t}!$ The exception is the last hidden state, $h_n$:

$\frac{\partial y}{\partial h_n} = W_{hy}$

We now have everything we need to finally implement BPTT and finish backprop():


In [0]:
class RNN:
  # A Vanilla Recurrent Neural Network.

  def __init__(self, input_size, output_size, hidden_size=64):
    # Weights
    self.Whh = randn(hidden_size, hidden_size) / 1000
    self.Wxh = randn(hidden_size, input_size) / 1000
    self.Why = randn(output_size, hidden_size) / 1000

    # Note: We're dividing by 1000 to reduce the initial variance of our weights.
    # This is not the best way to initialize weights, but it's simple and works for this example.

    # Biases
    self.bh = np.zeros((hidden_size, 1))
    self.by = np.zeros((output_size, 1))

  def forward(self, inputs):
    '''
    Perform a forward pass of the RNN using the given inputs.
    Returns the final output and hidden state.
    - inputs is an array of one hot vectors with shape (input_size, 1).
    '''
    h = np.zeros((self.Whh.shape[0], 1))

    self.last_inputs = inputs
    self.last_hs = { 0: h }

    # Perform each step of the RNN
    for i, x in enumerate(inputs):
      h = np.tanh(self.Wxh @ x + self.Whh @ h + self.bh)
      self.last_hs[i + 1] = h

    # Compute the output
    y = self.Why @ h + self.by

    return y, h
    

  def backprop(self, d_y, learn_rate=2e-2):
    '''
    Perform a backward pass of the RNN.
    - d_y (dL/dy) has shape (output_size, 1).
    - learn_rate is a float.
    '''
    n = len(self.last_inputs)

    # Calculate dL/dWhy and dL/dby.
    d_Why = d_y @ self.last_hs[n].T
    d_by = d_y

    # Initialize dL/dWhh, dL/dWxh, and dL/dbh to zero.
    d_Whh = np.zeros(self.Whh.shape)
    d_Wxh = np.zeros(self.Wxh.shape)
    d_bh = np.zeros(self.bh.shape)

    # Calculate dL/dh for the last h.
    d_h = self.Why.T @ d_y

    # Backpropagate through time.
    for t in reversed(range(n)):
      # An intermediate value: dL/dh * (1 - h^2)
      temp = ((1 - self.last_hs[t + 1] ** 2) * d_h)

      # dL/db = dL/dh * (1 - h^2)
      d_bh += temp

      # dL/dWhh = dL/dh * (1 - h^2) * h_{t-1}
      d_Whh += temp @ self.last_hs[t].T

      # dL/dWxh = dL/dh * (1 - h^2) * x
      d_Wxh += temp @ self.last_inputs[t].T

      # Next dL/dh = dL/dh * (1 - h^2) * Whh
      d_h = self.Whh @ temp

    # Clip to prevent exploding gradients.
    for d in [d_Wxh, d_Whh, d_Why, d_bh, d_by]:
      np.clip(d, -1, 1, out=d)

    # Update weights and biases using gradient descent.
    self.Whh -= learn_rate * d_Whh
    self.Wxh -= learn_rate * d_Wxh
    self.Why -= learn_rate * d_Why
    self.bh -= learn_rate * d_bh
    self.by -= learn_rate * d_by

*   We’ve merged $\frac{\partial L}{\partial y} * \frac{\partial y}{\partial h}$ into $\frac{\partial L}{\partial h}$ for convenience.
*   We’re constantly updating $d_h$ variable that holds the most recent $\frac{\partial L}{\partial h_{t+1}}$, which we need to calculate $\frac{\partial L}{\partial h_t} $.





let’s test our RNN!

In [0]:
import random

def processData(data, backprop=True):
  '''
  Returns the RNN's loss and accuracy for the given data.
  - data is a dictionary mapping text to True or False.
  - backprop determines if the backward phase should be run.
  '''
  items = list(data.items())
  random.shuffle(items)

  loss = 0
  num_correct = 0

  for x, y in items:
    inputs = createInputs(x)
    target = int(y)

    # Forward
    out, _ = rnn.forward(inputs)
    probs = softmax(out)

    # Calculate loss / accuracy
    loss -= np.log(probs[target])
    num_correct += int(np.argmax(probs) == target)

    if backprop:
      # Build dL/dy
      d_L_d_y = probs
      d_L_d_y[target] -= 1

      # Backward
      rnn.backprop(d_L_d_y)

  return loss / len(data), num_correct / len(data)

In [0]:
# Training loop
for epoch in range(1000):
  train_loss, train_acc = processData(train_data)

  if epoch % 100 == 99:
    print('--- Epoch %d' % (epoch + 1))
    print('Train:\tLoss %.3f | Accuracy: %.3f' % (train_loss, train_acc))

    test_loss, test_acc = processData(test_data, backprop=False)
    print('Test:\tLoss %.3f | Accuracy: %.3f' % (test_loss, test_acc))

--- Epoch 100
Train:	Loss 0.688 | Accuracy: 0.569
Test:	Loss 0.699 | Accuracy: 0.500
--- Epoch 200
Train:	Loss 0.672 | Accuracy: 0.638
Test:	Loss 0.717 | Accuracy: 0.550
--- Epoch 300
Train:	Loss 0.540 | Accuracy: 0.724
Test:	Loss 0.867 | Accuracy: 0.500
--- Epoch 400
Train:	Loss 0.429 | Accuracy: 0.776
Test:	Loss 0.770 | Accuracy: 0.550
--- Epoch 500
Train:	Loss 0.334 | Accuracy: 0.879
Test:	Loss 0.938 | Accuracy: 0.550
--- Epoch 600
Train:	Loss 0.097 | Accuracy: 0.948
Test:	Loss 0.256 | Accuracy: 0.850
--- Epoch 700
Train:	Loss 0.012 | Accuracy: 1.000
Test:	Loss 0.037 | Accuracy: 1.000
--- Epoch 800
Train:	Loss 0.005 | Accuracy: 1.000
Test:	Loss 0.013 | Accuracy: 1.000
--- Epoch 900
Train:	Loss 0.003 | Accuracy: 1.000
Test:	Loss 0.008 | Accuracy: 1.000
--- Epoch 1000
Train:	Loss 0.002 | Accuracy: 1.000
Test:	Loss 0.007 | Accuracy: 1.000


In [0]:
inputs = createInputs('i am very good')
out, h = rnn.forward(inputs)
probs = softmax(out)
print(probs) 

[[0.02143507]
 [0.97856493]]
